In [12]:
import os
import sys
import numpy as np
from pprint import pprint
from tqdm import tqdm
import pandas as pd
from datetime import datetime
from importlib import reload
import math
from IPython.display import display, Markdown
from datetime import datetime

sys.path.insert(0, os.path.abspath('../tesi_sabella'))
# pd.options.display.max_rows = 999

import pyfluxc.pyfluxc as flux
import data_generator as generator
reload(flux)
reload(generator)

<module 'data_generator' from '/Users/swedishhdcamera/Desktop/Projects/tesi_sabella/tesi_sabella/data_generator.py'>

In [13]:
# Constants ----- #
BUCKET = 'CIC_IDS_2017_Friday'
PULL_EVERY_X_MINUTES = 15
PULL_DELTA_TIME = pd.Timedelta(minutes=PULL_EVERY_X_MINUTES)

TCPREPLAY_START = {
    "Monday": "2020-04-04T17:19:06.730097",
    "Tuesday": "2020-04-05T14:09:30.933025",
    "Wednesday": "2020-04-06T08:46:54.338920",
    "Thursday": "2020-04-07T08:08:37.127602808",
    "Friday": "2020-04-07T22:41:05.814784058"}
TCPREPLAY_START = { k: pd.to_datetime(s, format='%Y-%m-%dT%H:%M:%S.%f') for k, s in TCPREPLAY_START.items() }

CIC_IDS_2017_START = {
    "Monday": "2017-07-03T08:55:58.598308",
    "Tuesday": "2017-07-04T08:53:32.364079",
    "Wednesday": "2017-07-05T08:42:42.084372",
    "Thursday": "2017-07-06T08:58:58.492265",
    "Friday": "2017-07-07T08:59:39.599128"}
CIC_IDS_2017_START = { k: pd.to_datetime(s, format='%Y-%m-%dT%H:%M:%S.%f') for k, s in CIC_IDS_2017_START.items() }

In [14]:
# Functions ----- #
def mprint(s):
    display(Markdown(s))

# Generating time series

We pull only the time range in which samples are available

In [16]:
fclient = flux.FluxClient(); 
start, end = fclient.bucket_timerange(BUCKET)
print(f"first sample: {start}\nlast sample: {end}\nTotal: {end-start}")

first sample: 2020-04-07 22:40:35
last sample: 2020-04-08 07:10:55
Total: 0 days 08:30:20


In [17]:
cicids2017 = generator.CICIDS2017(BUCKET, '15s', fclient, start)

Extract data from influxdb in time intervals due to influx kill signal

In [ ]:
i = 1
pull_start = start
ts = []
while True:
    pull_end = pull_start + PULL_DELTA_TIME
    
    print("------------")
    print(f"first sample: {pull_start}\nlast sample: {pull_end}\nTotal: {pull_end-pull_start}")
    print("------------")
    
    # TODO: return tuple
    t_now = datetime.now()
    pull_start, new_samples = cicids2017.pull(stop=pull_end)
    ts.append(datetime.now() - t_now)
    if new_samples is None:
        break

mprint(f"**Mean pulling time:** {np.mean(ts)}")

------------
first sample: 2020-04-07 22:40:35
last sample: 2020-04-07 22:55:35
Total: 0 days 00:15:00
------------


In [7]:
DIFF = False
cicids2017_df = cicids2017.to_pandas(DIFF)

# Setting correct timing before saving
day = BUCKET.split("_")[-1]
delta = TCPREPLAY_START[day] - CIC_IDS_2017_START[day]
new_delta_time = (cicids2017_df.index.levels[2] - delta)
cicids2017_df.index.set_levels(new_delta_time, level="_time", inplace=True)

cicids2017_df.to_pickle(f"./{BUCKET}{'_no_delta' if not DIFF else ''}.pkl")

In [ ]:
def cest_to_utc(x):
    return x.tz_localize(tz='CET').tz_convert('UTC').tz_localize(tz=None)
    
TCPREPLAY_START_UTC = { k: cest_to_utc(x) for k, x in TCPREPLAY_START.items() }
for k, v in TCPREPLAY_START_UTC.items():
    print(v.strftime(format='%Y-%m-%dT%H:%M:%S.%f'))